<a href="https://colab.research.google.com/github/goldapplemango/ai-lotto/blob/main/filter_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# 1. 데이터 로드 및 전처리
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

def load_and_preprocess_data(data_source):
    """
    데이터를 로드하고 전처리하는 함수
    - 결측치 처리 및 이상치 제거 추가
    """
    data = pd.read_csv(data_source)
    data = data.dropna()  # 결측치 처리
    data = data[(data['번호1'] <= 45) & (data['번호2'] <= 45) & (data['번호3'] <= 45) &
                (data['번호4'] <= 45) & (data['번호5'] <= 45) & (data['번호6'] <= 45)]  # 이상치 제거
    data = data.sort_values(by='회차', ascending=False)
    return data

import pandas as pd

def load_data_from_url(url):
    """
    주어진 URL에서 CSV 데이터를 로드하고 전처리하여 반환하는 함수.
    """
    # 데이터 로드
    data = pd.read_csv(url)

    # 로드한 데이터 확인 (일부만 출력)
    print("데이터 미리보기:")
    print(data.head())

    # 필요한 데이터만 추출 (여기서 '번호1' ~ '번호6' 열이 있다고 가정)
    return data[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']]

# 실데이터 로드
# url = "https://raw.githubusercontent.com/goldapplemango/ai-lotto/refs/heads/main/lotto_data11.csv"
#lotto_data = load_data_from_url(url)

def analyze_data(data):
    """
    로드된 데이터에서 번호의 빈도수를 분석하여 가장 자주 등장한 번호를 찾는 함수.
    """
    all_numbers = data.values.flatten()
    frequency = pd.Series(all_numbers).value_counts().sort_index()

    print("빈도수 분석 결과:")
    print(frequency)
    return frequency

# 번호 빈도수 분석
# frequency = analyze_data(lotto_data)

# 2. 필터링 모듈
def advanced_filtering(data, min_frequency=10, exclude_recent=3):
    """
    데이터에서 번호 후보군을 생성하는 고급 필터링
    - min_frequency: 최소 출현 빈도
    - exclude_recent: 최근 n회 동안 등장한 번호 배제
    """
    # 번호별 빈도 계산
    all_numbers = data[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
    frequency = pd.Series(all_numbers).value_counts()

    # 최소 빈도수 조정
    filtered_numbers = frequency[frequency >= min_frequency].index.tolist()

    # 최근 번호 제외를 더 유연하게 적용 (최근 3회 이내 배제, 5회 이내로 제한)
    recent_numbers = data.iloc[:exclude_recent][['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
    filtered_numbers = [num for num in filtered_numbers if num not in recent_numbers]

    # 빈도가 낮거나 최근에 나온 번호를 필터링하지만, 일부 번호는 남겨둔다.
    return filtered_numbers

def filter_candidates(data, min_frequency=5, exclude_recent=5):
    """
    번호의 빈도수 및 최근 등장한 번호들을 기반으로 필터링된 후보 번호를 반환하는 함수.
    """
    # 빈도수 계산
    all_numbers = data.values.flatten()
    frequency = pd.Series(all_numbers).value_counts()

    # 빈도수가 min_frequency 이상인 번호 필터링
    filtered = frequency[frequency >= min_frequency].index.tolist()

    # 최근 등장한 번호를 제외 (예시로 마지막 5주간 등장한 번호를 제외)
    recent_numbers = data.tail(exclude_recent).values.flatten()
    filtered = [num for num in filtered if num not in recent_numbers]

    print("필터링된 후보 번호:", filtered)
    return filtered

## 필터링된 번호 후보
# filtered_candidates = filter_candidates(lotto_data)


# 3. 유전 알고리즘 모듈
from geneticalgorithm import geneticalgorithm as ga

def fitness_function(X):
    """
    유전 알고리즘의 적합도 함수
    - X: 번호 조합 (6개의 숫자)
    """
    score = 0
    for num in X:
        score += weights.get(int(num), 0)  # 번호별 가중치 점수

    # 패널티: 연속 번호가 너무 많으면 점수 감소
    if any(abs(X[i] - X[i+1]) == 1 for i in range(len(X) - 1)):
        score -= 10

    # 중복 방지
    if len(set(X)) < 6:
        score -= 50

    return -score  # 최적화는 최소화가 목표

import random

from geneticalgorithm import geneticalgorithm as ga

def optimize_combinations(filtered_numbers):
    """
    유전 알고리즘을 사용하여 최적 조합을 생성하는 함수.
    """
    # 유전 알고리즘의 변수 범위 정의
    varbound = [(1, 45)] * 6  # 번호 범위 (1~45)

    # 유전 알고리즘 모델 설정
    model = ga(function=fitness_function,
               dimension=6,  # 6개의 번호
               variable_type='int',
               variable_boundaries=varbound,
               n_pop=10,  # 초기 인구 수
               max_num_of_generations=50,  # 세대 수
               parent_selection_type='rws',  # 부모 선택 방식 (룰렛 휠 선택)
               crossover_type='uniform',  # 교차 방식
               crossover_probability=0.7,  # 교차 확률
               mutation_type='random',  # 돌연변이 방식
               mutation_probability=0.1,  # 돌연변이 확률
               elitism=True)  # 엘리트 기법 사용

    model.run()  # 유전 알고리즘 실행
    return model.output_dict['variable']

# 최적 조합 생성
# best_combination = optimize_combinations(filtered_candidates)
# print("최적 조합:", best_combination)

# 4. 시뮬레이션 모듈
def simulate_combinations(data, combinations):
    """
    생성된 조합들을 시뮬레이션하여 성공 확률을 계산하는 함수.
    """
    # 시뮬레이션을 위해 번호 출현 빈도와 비교
    all_numbers = data.values.flatten()
    success_count = 0

    for combo in combinations:
        if all(num in all_numbers for num in combo):
            success_count += 1

    # 성공 확률 계산
    success_rate = success_count / len(combinations)
    return success_rate

# 최적화된 번호 조합에 대한 시뮬레이션
# generated_combinations = [best_combination]
# success_rate = simulate_combinations(lotto_data, generated_combinations)
# print(f"시뮬레이션 성공률: {success_rate * 100:.2f}%")

import matplotlib.pyplot as plt
def visualize_results(frequency):
    """
    로또 번호 빈도수를 시각화하는 함수
    """
    frequency.sort_values(ascending=False).plot(kind='bar', figsize=(12, 6))
    plt.title("로또 번호 출현 빈도")
    plt.xlabel("번호")
    plt.ylabel("빈도")
    plt.show()

from multiprocessing import Pool

def parallel_simulation(data, combinations):
    """
    병렬로 시뮬레이션을 실행하여 성능 향상
    """
    with Pool() as pool:
        results = pool.starmap(simulate_combinations, [(data, [combo]) for combo in combinations])
    return results


# 5. 메인 실행 워크플로우
def main():
    # 데이터 로드
    act_path ="/content/drive/MyDrive/lotto7/lotto_data11.csv"
    data = load_and_preprocess_data(act_path)


    # 데이터 분석 및 필터링
    global frequency
    all_numbers = data[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
    frequency = pd.Series(all_numbers).value_counts()

    # 필터링된 후보 번호 출력
    filtered_candidates = advanced_filtering(data, min_frequency=10, exclude_recent=5)
    print("필터링된 후보 번호:", filtered_candidates)

    # 유전 알고리즘을 통해 최적 조합 생성
    best_combination = optimize_combinations(filtered_candidates)
    print("최적 조합:", best_combination)

    # 최적화된 번호 조합을 시뮬레이션하여 성공 확률 계산
    generated_combinations = [best_combination]
    success_rate = simulate_combinations(data, generated_combinations)
    print(f"시뮬레이션 성공률: {success_rate * 100:.2f}%")

    # 결과 시각화
    visualize_results(frequency)

if __name__ == "__main__":
    main()

#

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
필터링된 후보 번호: [14, 18, 13, 45, 20, 1, 21, 38, 39, 35, 30, 42, 8, 5, 29, 25, 41, 23, 22, 32, 9]


TypeError: geneticalgorithm.__init__() got an unexpected keyword argument 'n_pop'

In [2]:

!pip install matplotlib pandas geneticalgorithm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15076 sha256=9a074f0b8d4a5d0b541a5d0c2e9e0d4bdfcde6f66202f9487a2dda07cee13227
  Stored in directory: /root/.cache/pip/wheels/3f/83/19/b5552bb9630e353f7c5b15be44bf10900afe1abbbfcf536afd
Successfully built func-timeout
